<a href="https://colab.research.google.com/github/yuxuanbrandeis/Julex/blob/main/Slice_by_paragraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install secedgar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of aiohttp to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for aiohttp: filename=aiohttp-3.7.4.post0-cp310-cp310-linux_x86_64.whl size=1416356 sha256=17dfbf0c8d568ee23a5373ca8271c217f188b0a581b4ee6fd81e7db1535fdd06
  Stored in directory: /root/.cache/pip/wheels/fc/84/e0/52113c57eb9b09b6b187a0f369eaad6fc7fc64bb7247c83b89
Successfully built aiohttp
  Attempting uninst

In [3]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00


In [4]:
pip install torch torchvision torchaudio

In [5]:
import os
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
nltk.download('vader_lexicon')
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [6]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.4 MB/s eta 0:00:00


In [9]:

def extract_mda_section(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Pattern to match the start of the MD&A section
    pattern = r"(?i)\bitem\s*2\b"

    # Find the start of the MD&A section in the content
    start_match = re.search(pattern, content)

    if start_match:
        start_index = start_match.start()

        # Find the end of the paragraph
        end_match = re.search(r"\n\s*\n", content[start_index:])

        if end_match:
            end_index = end_match.end() + start_index
        else:
            # If no explicit end found, consider the end of the content
            end_index = len(content)

        # Extract the MD&A section paragraph
        mda_paragraph = content[start_index:end_index]

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(mda_paragraph, 'html.parser')

        # Remove tables and graphs from the HTML
        for table in soup.find_all('table'):
            table.extract()
        for graph in soup.find_all('img'):
            graph.extract()

        # Extract the plain text without HTML tags
        mda_text = soup.get_text()
        return mda_text.strip()

    else:
        print("MD&A section not found in the HTML file")
        return None


In [10]:
# import a test folder
test_folder_path = "/content/drive/MyDrive/Julex/September/20220901/1090872/0001090872-22-000017.txt"

In [11]:
test_mda = extract_mda_section(test_folder_path)
print(test_mda)

Item 2.Unregistered Sales of Equity Securities and Use of Proceeds61 Item 6.Exhibits62Signature  632Table of ContentsPART I— FINANCIAL INFORMATION ITEM 1. CONDENSED CONSOLIDATED FINANCIAL STATEMENTS (UNAUDITED) AGILENT TECHNOLOGIES, INC.CONDENSED CONSOLIDATED STATEMENT OF OPERATIONS(in millions, except per share amounts)(Unaudited)  The accompanying notes are an integral part of these condensed consolidated financial statements.3Table of ContentsAGILENT TECHNOLOGIES, INC.CONDENSED CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME (LOSS)(in millions)(Unaudited)The accompanying notes are an integral part of these condensed consolidated financial statements.4Table of ContentsAGILENT TECHNOLOGIES, INC.CONDENSED CONSOLIDATED BALANCE SHEET(in millions, except par value and share amounts)(Unaudited) The accompanying notes are an integral part of these condensed consolidated financial statements.5Table of ContentsAGILENT TECHNOLOGIES, INC.CONDENSED CONSOLIDATED STATEMENT OF CASH FLOWS(in millions

In [12]:
# slice this section by paragraph
from nltk.tokenize import sent_tokenize

from nltk.tokenize import sent_tokenize, word_tokenize

def slice_text_by_paragraphs(text):
    paragraphs = text.split("\n\n")  # Assuming paragraphs are separated by two newline characters

    segmented_text = []

    for paragraph in paragraphs:
        # Tokenize the paragraph into words
        words = word_tokenize(paragraph)

        # Check the length of the paragraph in words
        if len(words) > 300:
            # Split the paragraph into segments of 512 words or less
            num_segments = (len(words) // 300) + 1
            segment_size = len(words) // num_segments

            for i in range(num_segments):
                start = i * segment_size
                end = (i + 1) * segment_size
                segment = words[start:end]
                segmented_text.append(" ".join(segment))
        else:
            segmented_text.append(paragraph)

    return segmented_text




In [13]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
test_paragraph = slice_text_by_paragraphs(test_mda)
print(test_paragraph)

["Item 2.Unregistered Sales of Equity Securities and Use of Proceeds61 Item 6.Exhibits62Signature 632Table of ContentsPART I— FINANCIAL INFORMATION ITEM 1 . CONDENSED CONSOLIDATED FINANCIAL STATEMENTS ( UNAUDITED ) AGILENT TECHNOLOGIES , INC.CONDENSED CONSOLIDATED STATEMENT OF OPERATIONS ( in millions , except per share amounts ) ( Unaudited ) The accompanying notes are an integral part of these condensed consolidated financial statements.3Table of ContentsAGILENT TECHNOLOGIES , INC.CONDENSED CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME ( LOSS ) ( in millions ) ( Unaudited ) The accompanying notes are an integral part of these condensed consolidated financial statements.4Table of ContentsAGILENT TECHNOLOGIES , INC.CONDENSED CONSOLIDATED BALANCE SHEET ( in millions , except par value and share amounts ) ( Unaudited ) The accompanying notes are an integral part of these condensed consolidated financial statements.5Table of ContentsAGILENT TECHNOLOGIES , INC.CONDENSED CONSOLIDATED STATE

In [17]:
def write_result_to_file(result, file_path):
    with open(file_path, 'w') as file:
        file.write(result)

result = test_mda
file_path = "/content/drive/MyDrive/Julex/result.txt"

write_result_to_file(result, file_path)

In [18]:
from transformers import pipeline
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

In [19]:

def perform_sentiment_analysis(text):
    # Initialize FinBERT sentiment analysis pipeline
    sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert", tokenizer="ProsusAI/finbert")

    # Initialize NLTK SentimentIntensityAnalyzer
    nltk_sentiment_analyzer = SentimentIntensityAnalyzer()

    # Initialize the result list to store sentiment scores
    sentiment_scores = []

    for paragraph in text:
        # Split long paragraphs into smaller segments if needed
        if len(paragraph.split()) > 512:
            segmented_paragraphs = split_long_paragraph(paragraph)
        else:
            segmented_paragraphs = [paragraph]

        # Perform sentiment analysis for each segmented paragraph
        for segmented_paragraph in segmented_paragraphs:
            # Perform sentiment analysis using FinBERT
            finbert_result = sentiment_pipeline(segmented_paragraph)[0]

            # Perform sentiment analysis using NLTK SentimentIntensityAnalyzer
            nltk_result = nltk_sentiment_analyzer.polarity_scores(segmented_paragraph)
            nltk_sentiment = "positive" if nltk_result["compound"] >= 0 else "negative"

            # Store the sentiment scores
            sentiment_scores.append({
                "finbert_sentiment": finbert_result["label"],
                "finbert_score": finbert_result["score"],
                "nltk_sentiment": nltk_sentiment,
                "nltk_score": nltk_result["compound"]
            })

    return sentiment_scores
    print(sentiment_scores)

def split_long_paragraph(paragraph):
    # Tokenize paragraph into words
    words = word_tokenize(paragraph)

    # Split paragraph into smaller segments
    max_segment_length = 300  # Maximum length for each segment
    segmented_paragraphs = []

    i = 0
    while i < len(words):
        segment = " ".join(words[i:i+max_segment_length])
        segmented_paragraphs.append(segment)
        i += max_segment_length

    return segmented_paragraphs





In [20]:
test_paragraph_score = perform_sentiment_analysis(test_paragraph)
print(test_paragraph_score)

[{'finbert_sentiment': 'neutral', 'finbert_score': 0.9458103775978088, 'nltk_sentiment': 'positive', 'nltk_score': 0.9329}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.9394246935844421, 'nltk_sentiment': 'positive', 'nltk_score': 0.6652}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.931207537651062, 'nltk_sentiment': 'positive', 'nltk_score': 0.7269}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.9505639672279358, 'nltk_sentiment': 'positive', 'nltk_score': 0.171}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.939984917640686, 'nltk_sentiment': 'positive', 'nltk_score': 0.9919}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.929978609085083, 'nltk_sentiment': 'positive', 'nltk_score': 0.9941}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.9505609273910522, 'nltk_sentiment': 'positive', 'nltk_score': 0.8591}, {'finbert_sentiment': 'neutral', 'finbert_score': 0.9293397665023804, 'nltk_sentiment': 'positive', 'nltk_score': 0.8555}, {'finbert_sentiment

In [22]:
def calculate_average_score(sentiment_scores):
    num_paragraphs = len(sentiment_scores)
    finbert_scores = [score["finbert_score"] for score in sentiment_scores]
    nltk_scores = [score["nltk_score"] for score in sentiment_scores]

    finbert_average_score = sum(finbert_scores) / num_paragraphs
    nltk_average_score = sum(nltk_scores) / num_paragraphs

    if finbert_average_score >= 0.1:
        overall_sentiment = "positive"
    elif finbert_average_score <= -0.1:
        overall_sentiment = "negative"
    else:
        overall_sentiment = "neutral"

    return {
        "finbert_average_score": finbert_average_score,
        "nltk_average_score": nltk_average_score,
        "overall_sentiment": overall_sentiment
    }


In [23]:
test_score = calculate_average_score(test_paragraph_score)
print(test_score)

{'finbert_average_score': 0.7938763693321583, 'nltk_average_score': 0.5174480620155036, 'overall_sentiment': 'positive'}
